In [1]:
import os
import glob
import pickle
import sys
# If your current working directory is the notebooks directory, use this:
library_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'library'))
sys.path.append(library_path)

import pandas as pd
import datetime as dt
from datetime import date, datetime
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

today = date.today().strftime("%d%m%Y")
today_day = pd.to_datetime('today').normalize()

today = "12062024"

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/blueswatch_open/data/"
filepath = datapath + f"export_bw_{today}"


In [2]:
# passive + ema_data
file_pattern = os.path.join(filepath, "daily_part*.csv")
file_list = glob.glob(file_pattern)
file_list.sort()

df_complete = pd.concat((pd.read_csv(f, encoding="latin-1", low_memory=False) for f in file_list), ignore_index=True)

In [3]:
df_complete["customer"] = df_complete["customer"].str[:4]


In [4]:
df_complete.customer.nunique()

1979

In [5]:
# Step 1: Calculate the percentage of customers with at least one entry per data type

# Group by customer and type, and flag presence of each type
customer_type_presence = df_complete.groupby(['customer', 'type']).size().reset_index(name='count')

# Pivot table to get presence (1 if present, NaN if not) of each type per customer
customer_type_presence = customer_type_presence.pivot(index='customer', columns='type', values='count').notna()

# Calculate percentage of customers with at least one entry for each data type
type_percentage = customer_type_presence.mean() * 100
percentage_df = type_percentage.reset_index(name='Percentage')
percentage_df.columns = ['DataType', 'Percentage']

# Step 2: Aggregate sources for each data type and count the number of sources

# Group by 'type' and aggregate sources into a list
sources_df = df_complete.groupby('type')['source'].unique().reset_index()
sources_df.columns = ['DataType', 'Sources']
sources_df['Sources'] = sources_df['Sources'].apply(list)
sources_df['NumberOfSources'] = sources_df['Sources'].apply(len)

# Step 3: Merge the two DataFrames

# Merge the percentage_df and sources_df DataFrames on 'DataType'
combined_df = pd.merge(percentage_df, sources_df, on='DataType')

# Sort by the number of sources and then by percentage
combined_df = combined_df.sort_values(by=['NumberOfSources', 'Percentage'], ascending=[False, False])

combined_df

,DataType,Percentage,Sources,NumberOfSources
69,Steps,97.675594,"[Apple, Nokia, Garmin, Fitbit, GoogleFit, Oura...",8
15,BurnedCalories,94.744821,"[Apple, Nokia, Garmin, Fitbit, GoogleFit, Oura...",7
1,ActiveDuration,85.346134,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Go...",7
68,SleepStartTime,82.566953,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Sa...",7
60,SleepEndTime,82.516422,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Sa...",7
...,...,...,...,...
49,SleepAutonomicNervousSystemCharge,0.960081,[Polar],1
50,SleepAutonomicNervousSystemChargeStatus,0.960081,[Polar],1
71,UVindex,0.859020,[Samsung],1
67,SleepRatingReport,0.656897,[Polar],1


In [21]:
combined_df.head(50)

,DataType,Percentage,Sources,NumberOfSources
69,Steps,97.675594,"[Apple, Nokia, Garmin, Fitbit, GoogleFit, Oura...",8
15,BurnedCalories,94.744821,"[Apple, Nokia, Garmin, Fitbit, GoogleFit, Oura...",7
1,ActiveDuration,85.346134,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Go...",7
68,SleepStartTime,82.566953,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Sa...",7
60,SleepEndTime,82.516422,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar, Sa...",7
11,BikeDuration,31.531076,"[Apple, Fitbit, Garmin, Nokia, GoogleFit, Pola...",7
0,ActiveBurnedCalories,90.197069,"[Apple, Garmin, Fitbit, Oura, Polar, Samsung]",6
61,SleepInBedDuration,82.769075,"[Apple, Nokia, Garmin, Fitbit, Oura, Samsung]",6
57,SleepDuration,82.364831,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar]",6
52,SleepAwakeDuration,73.825164,"[Apple, Nokia, Garmin, Fitbit, Oura, Polar]",6


In [6]:
import pandas as pd

# Assuming df_complete is your DataFrame and 'customer' is the column with customer IDs,
# 'type' is the column with data types, and 'source' is the column indicating sources.

# Group by 'source' and 'customer'
grouped = df_complete.groupby(['source', 'customer']).size().reset_index(name='count')

# For each source, find the number of unique customers
customer_counts = grouped.groupby('source')['customer'].nunique().reset_index(name='CustomerCount')

# Find the total number of unique customers in the entire dataset
total_customers = df_complete['customer'].nunique()

# Calculate the percentage of customers per source
customer_counts['Percentage'] = (customer_counts['CustomerCount'] / total_customers) * 100

# Sort by percentage in descending order
customer_counts = customer_counts.sort_values(by='Percentage', ascending=False)

customer_counts


,source,CustomerCount,Percentage
0,Apple,873,44.113188
2,Garmin,521,26.326427
1,Fitbit,413,20.869126
4,Nokia,94,4.749874
3,GoogleFit,79,3.991915
6,Polar,69,3.486609
7,Samsung,35,1.768570
5,Oura,19,0.960081


In [7]:

typelist = ["Steps", "BurnedCalories", "ActiveBurnedCalories", "HeartRateResting", "ActiveDuration", "SleepInBedDuration"
           , "SleepStartTime", "SleepEndTime", "SleepDuration", "FloorsClimbed", "SleepAwakeDuration", "CoveredDistance"]

In [8]:
df_complete = df_complete.loc[df_complete.type.isin(typelist)]

In [10]:
df_complete.loc[df_complete.type.isin(["SleepEndTime", "SleepStartTime"])]

,customer,source,day,type,valueType,doubleValue,longValue,booleanValue,dateValue,stringValue,generation,trustworthiness,medicalGrade,userReliability,chronologicalExactness,timezoneOffset,createdAt
8,02YI,Apple,2022-08-17,SleepEndTime,40,NaN,NaN,NaN,1.660704e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1660713934750
10,02YI,Apple,2022-08-17,SleepStartTime,40,NaN,NaN,NaN,1.660699e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1660713934750
20,02YI,Apple,2022-08-18,SleepEndTime,40,NaN,NaN,NaN,1.660858e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1660880407067
22,02YI,Apple,2022-08-18,SleepStartTime,40,NaN,NaN,NaN,1.660776e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1660798509668
32,02YI,Apple,2022-08-19,SleepEndTime,40,NaN,NaN,NaN,1.660879e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1660889852943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16609663,x8Vm,Apple,2022-09-28,SleepStartTime,40,NaN,NaN,NaN,1.664329e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1664385967217
16609736,x8Vm,Apple,2022-10-21,SleepEndTime,40,NaN,NaN,NaN,1.666345e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1666361562729
16609737,x8Vm,Apple,2022-10-21,SleepStartTime,40,NaN,NaN,NaN,1.666309e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1666361562729
16609747,x8Vm,Apple,2022-10-22,SleepEndTime,40,NaN,NaN,NaN,1.666437e+12,NaN,NaN,NaN,NaN,NaN,NaN,120.0,1666436950348


In [11]:
# Create separate dataframes for each value column
double_value_df = df_complete.pivot_table(index=["customer", "source", "day"], columns="type", values="doubleValue")
long_value_df = df_complete.pivot_table(index=["customer", "source", "day"], columns="type", values="longValue")
date_value_df = df_complete.pivot_table(index=["customer", "source", "day"], columns="type", values="dateValue")

# Merge these dataframes back together
merged_df = double_value_df.join(long_value_df, lsuffix='_doubleValue', rsuffix='_longValue')\
                           .join(date_value_df, rsuffix='_dateValue')

# Reset index to make 'customer', 'source', and 'day' columns
merged_df.reset_index(inplace=True)
merged_df.day = pd.to_datetime(merged_df.day)


In [13]:
with open(datapath + f'/quest_data.pkl', 'rb') as file:
    df_quest = pickle.load(file)

In [14]:
merged_df = df_quest.merge(merged_df, on="customer")

In [15]:
filtered_df = merged_df[merged_df['day'] <= merged_df['quest_complete_day']]

In [17]:
merged_df.customer.nunique()

208

In [18]:
filtered_df.customer.nunique()

194

In [19]:
df_quest.customer.nunique()

292